In [ ]:
# 导入相关包
import math
import operator
import matplotlib.pyplot as plt
#python在读写matlab文件时常用到scipy.io文件
import scipy.io as sio
import openpyxl
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,BatchNormalization,LeakyReLU,Conv2DTranspose,Reshape,Conv2D,Dropout,Flatten,AveragePooling1D,Activation
import tensorflow as tf
import time
import os
import matplotlib
from tensorflow.keras.models import Model
import tensorflow.keras
from scipy.signal import savgol_filter
from numpy import *

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# os.chdir(r'D:\大麦粉数据')
x = np.load('trainx.npy',allow_pickle = True)
y = np.load('trainy.npy',allow_pickle = True)
axis_x = np.load('axis_x.npy',allow_pickle = True)
#定义真实光谱数据
wave = x.reshape(-1,2500)
wave = wave.astype('float64')
print(wave.shape)

In [ ]:
def PlotSpectrum(spec,axis_x):
    plt.figure(figsize=(5.2, 3.1), dpi=300)
    x = axis_x
    for i in range(spec.shape[0]):
        plt.plot(x, spec[i, :], linewidth=0.6)
    fonts = 8
    plt.xlim(x.min(),x.max())
    plt.ylim(0, 1)
    # 设置刻度字体大小
    plt.xlabel('Wavelength (nm)', fontsize=fonts)
    plt.ylabel('absorbance (AU)', fontsize=fonts)
    plt.yticks(fontsize=fonts)
    plt.xticks(fontsize=fonts)
    #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
    #调整方格
    plt.tight_layout(pad=0.3)
    #网格线设置
    plt.grid(True)
    return plt
PlotSpectrum(wave,axis_x)

In [ ]:
generator = tf.keras.models.load_model('MSG_generatorLast.h5')
discriminator = tf.keras.models.load_model('MSG_discriminatorLast.h5')

In [ ]:
discriminator.summary()

In [ ]:
discriminator1 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=2).output)# 输出为1800，第2层

discriminator2 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=5).output)# 输出为1800，第8层

discriminator3 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=11).output)# 输出为1800，第8层

discriminator4 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=14).output)# 输出为1800，第8层

discriminator5 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=8).output)# 输出为500，最内层

In [ ]:
# 定义超参数
epochs = 6
#定义一些常数
lambda_k1=0.0002
#k的初值设为：0
k = tf.Variable(0.005,dtype=tf.float64)
#设置gamma(高多样性好)
gamma = tf.Variable(0.7,dtype=tf.float64)

In [ ]:
# 定义外层损失函数
def L1(wave):
    reconstituted_wave = discriminator(wave)# 输出D(G(x))
    reconstituted_wave = tf.cast(reconstituted_wave,dtype=tf.float64)
    loss1 = tf.reduce_mean(tf.abs(wave - reconstituted_wave))
    return loss1
def L2(noise):
    generated_wave = generator(noise)# 通过生成器生成光谱generated_Spectrum
    generated_wave = tf.cast(generated_wave,dtype=tf.float64)
    reconstituted_generatedWave = discriminator(generated_wave)# 输出D(G(z))
    reconstituted_generatedWave = tf.cast(reconstituted_generatedWave,dtype=tf.float64)
    #作差-取绝对-求平均值(L(G(x)))
    loss2 = tf.reduce_mean(tf.abs(reconstituted_generatedWave - generated_wave))
    return loss2
def outermost_of_G(noise):# 最外层的损失
    loss = L2(noise)
    return loss
def outermost_of_D(wave, noise, k):# 最外层的损失
    loss1 = L1(wave)
    loss2 = L2(noise)
    loss = loss1 - k*loss2
    return loss

In [ ]:
# 定义内层的损失函数（ReMix实现方式，通过分别给真实光谱X1,X2计算损失值，并给予不同权重）
def mesosphere_L1(wave, model1, model2):# 输入真实光谱&每一层的判别模型
    reconstituted_wave1 = model1(wave)# 输出D1(G(x))
    reconstituted_wave1 = tf.cast(reconstituted_wave1,dtype=tf.float64)
    reconstituted_wave2 = model2(wave)# 输出D2(G(x))
    reconstituted_wave2 = tf.cast(reconstituted_wave2,dtype=tf.float64)    
    # 输出D1(G(x)) - D2(G(x))
    loss1 = tf.reduce_mean(tf.abs(reconstituted_wave1 - reconstituted_wave2))
    return loss1
def mesosphere_L2(noise, model1, model2):
    generated_wave = generator(noise)# 通过生成器生成光谱generated_Spectrum
    generated_wave = tf.cast(generated_wave,dtype=tf.float64)
    reconstituted_generatedWave1 = model1(generated_wave)# 输出D1(G(z))
    reconstituted_generatedWave1 = tf.cast(reconstituted_generatedWave1,dtype=tf.float64)
    reconstituted_generatedWave2 = model2(generated_wave)# 输出D2(G(z))
    reconstituted_generatedWave2 = tf.cast(reconstituted_generatedWave2,dtype=tf.float64)
    #作差-取绝对-求平均值(L(G(x)))
    loss2 = tf.reduce_mean(tf.abs(reconstituted_generatedWave1 - reconstituted_generatedWave2))  
    return loss2
def mesosphere_of_G(noise, model1, model2):# 内层的损失
    loss = mesosphere_L2(noise, model1, model2)
    return loss
def mesosphere_of_D(noise, wave, model1, model2, k):# 内层的损失
    loss1 = mesosphere_L1(wave, model1, model2)
    loss2 = mesosphere_L2(noise, model1, model2)
    loss = loss1 - k*loss2
    return loss

## 定义的ReMix：外层与MSGGAN一致，内层将以标签距离设置权重

In [ ]:
# 定义ReMix损失函数（最内层损失）
def Innermost_L(wave1, wave2, noise, model, label, nearest_label):# 输入真实光谱&每一层的判别模型
    generated_wave = generator(noise)# 通过生成器生成光谱generated_Spectrum
    generated_wave = tf.cast(generated_wave,dtype=tf.float64)
    reconstituted_wave = model(generated_wave)# 输出DInner(G(generated_wave))
    reconstituted_wave = tf.cast(reconstituted_wave,dtype=tf.float64)
    reconstituted_wave1 = model(wave1)# 输出DInner(G(wave1))
    reconstituted_wave1 = tf.cast(reconstituted_wave1,dtype=tf.float64)
    reconstituted_wave2 = model(wave2)# 输出DInner(G(wave2))
    reconstituted_wave2 = tf.cast(reconstituted_wave2,dtype=tf.float64)
    loss1 = tf.reduce_mean(tf.abs(reconstituted_wave - reconstituted_wave1))
    loss2 = tf.reduce_mean(tf.abs(reconstituted_wave - reconstituted_wave2))
    alpha = Calculated_weight(label, nearest_label)
    loss = alpha*loss1 + (1 - alpha)*loss2
    return loss1
def Calculated_weight(label, nearest_label):
    # alpha代表的是权重，值越大，权重越高，生成的理化值与该标签越近
    differ = np.absolute(nearest_label - label)# 相减赋予绝对值
    differSum = np.sum(differ)
    if (differ == 0).any() == True:
        if differ[0] == 0:# 该值为0时，则表明生成指标=第一个真实理化值，故权重要大
            alpha = 1# 返回的是第一个光谱的权重
            return alpha
        else:
            alpha = 0
            return alpha
    alpha = differ[1]/differSum
    return alpha# 返回的是第一个光谱的权重

In [ ]:
#定义总体进度函数
def global_process(wave, noise, gamma):
    loss1 = L1(wave)
    loss2 = L2(noise)
    balance = gamma*loss1 - loss2
    balance = tf.cast(balance,dtype=tf.float64)
    #总体进度定义
    global_goal = loss1 + tf.abs(balance)
    return global_goal, balance

In [ ]:
# 选取真实光谱曲线函数
# 找到n个最近邻
# 按照理化值寻找最近邻
def selectY(trainXset, trainYset, fakeY, n):# 真实y；生成y，需要的最近邻数量
    minusY = trainYset - fakeY# 差值
    minusY_2 = np.square(minusY)# 平方
    nearestIndex = minusY_2.argsort()# 从小到大排序
    nearestIndex = nearestIndex[0:n]
    nearestY = trainYset[nearestIndex]
    nearestX = trainXset[nearestIndex]
    nearestX = nearestX.astype('float64')
    return nearestX, nearestY# 输出n个最近邻的下标,最近邻X，最近邻Y

In [ ]:
# 制造一个批次的生成器输入
# 噪音（100）+二进制编码(100)+噪音（100）+二进制编码（100）+噪音（100）
# 噪音以独热编码的形式放入
def inputG(fakeY):
    noiseUnit1 = tf.random.uniform([1, 200])# 生成一个200维的随机tensor
    fakeY = fakeY*100
    hundredPlace = int(fakeY/100)
    tenPlace = int((fakeY%100)/10)
    onePlace = int((fakeY%100)%10)
    hundredOnehot = customize_Onehot(hundredPlace)
    tenOnehot = customize_Onehot(tenPlace)
    oneOnehot = customize_Onehot(onePlace)
    hundredOnehot = hundredOnehot*6
    tenOnehot = tenOnehot*3
    oneOnehot = oneOnehot*1
    hundredOnehot = np.array(hundredOnehot)
    tenOnehot = np.array(tenOnehot)
    oneOnehot = np.array(oneOnehot)
    finalOnthot = np.concatenate((hundredOnehot, tenOnehot, oneOnehot),axis=None)
    finalOnthot = finalOnthot.reshape(1,100)# 更改维度
    finalOnthot = tf.convert_to_tensor(finalOnthot,dtype=tf.float32)# 改变数据类型(tensor,float32)
    codeInput = tf.concat([finalOnthot, noiseUnit1[:,0:100], finalOnthot, noiseUnit1[:,100:200], finalOnthot],1)# 最后的G输入
    return codeInput
# 自定义十位数独热编码
def customize_Onehot(data):
    Onehot = []
    for i in range(10):
        if i == data:
            Onehot.append(1)
        else:
            Onehot.append(0)
    return Onehot

In [ ]:
# 在训练之前，确定需要得到的y值(120个：超过难以实现)
def generated_Sety(trainsetY):
    sortY = np.sort(trainsetY)# 对理化值进行排序
    generatedsetY = []
    for i in range(120):
        generatedY = sortY[i] + np.random.uniform(0,0.1)# 在每个y的基础上增加一个随机量
        generatedsetY.append(generatedY)
    return generatedsetY# 输出所要生成的y集合

In [ ]:
def image_train(nearestX, z, i, j, alpha):
    if i%30 == 0:# 每训练三十次，出一次训练图
        # 在训练中占比高的设置为红色，低则为蓝色
        generated_wave = generator(z)
        generated_wave = np.array(generated_wave)
        generated_wave = generated_wave.reshape(2500,1)
        if alpha >= 0.5:# 第一个真实光谱在训练中的权重占比较高
            print("这是第%s次训练，第%s张图"%(j, int(i/30)))
            plt.figure(figsize=(5.2, 3.1), dpi=300)# 设置画布
            fonts = 8
            plt.xlim(axis_x.min(),axis_x.max())
            plt.ylim(-0.1, 1)
            # 设置刻度字体大小
            plt.xlabel('Wavelength (nm)', fontsize=fonts)
            plt.ylabel('absorbance (AU)', fontsize=fonts)
            plt.yticks(fontsize=fonts)
            plt.xticks(fontsize=fonts)
            #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
            #调整方格
            plt.tight_layout(pad=0.25)
            #网格线设置
            plt.grid(True)
            plt.plot(axis_x, nearestX[0], linewidth = 0.6, c = 'red')
            plt.plot(axis_x, nearestX[1], linewidth = 0.6, c = 'blue')
            plt.plot(axis_x, generated_wave, linewidth = 0.6, c = 'k')
            plt.show()
        else:
            print("这是第%s次训练，第%s张图"%(j, int(i/30)))
            plt.figure(figsize=(5.2, 3.1), dpi=300)# 设置画布
            fonts = 8
            plt.xlim(axis_x.min(),axis_x.max())
            plt.ylim(-0.1, 1)
            # 设置刻度字体大小
            plt.xlabel('Wavelength (nm)', fontsize=fonts)
            plt.ylabel('absorbance (AU)', fontsize=fonts)
            plt.yticks(fontsize=fonts)
            plt.xticks(fontsize=fonts)
            #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
            #调整方格
            plt.tight_layout(pad=0.25)
            #网格线设置
            plt.grid(True)
            plt.plot(axis_x, nearestX[0], linewidth = 0.6, c = 'blue')
            plt.plot(axis_x, nearestX[1], linewidth = 0.6, c = 'red')
            plt.plot(axis_x, generated_wave, linewidth = 0.6, c = 'k')
            plt.show()

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)

In [ ]:
os.chdir(r'D:\Major_revision\Major_RevisondataSet')
x = np.load('trainx.npy',allow_pickle = True)
y = np.load('trainy.npy',allow_pickle = True)
# 将生成的y在真实y随机选择
real_y = y
generated_x = np.load('XfromX1.npy',allow_pickle = True)#如果文件路径末尾没有扩展名.npy，该扩展名会被自动加上
generated_y = np.load('YfromY1.npy',allow_pickle = True)
generated_y = generated_y.reshape(-1)
x = np.vstack((x,generated_x))
y = np.hstack((y,generated_y))

In [ ]:
generated_Yset = []
generated_spectrumSet = []
def Generated_spectrum():
    global k
    j = 1
    for epoch in range(epochs):# 定义总训练次数
        generatedSetY = generated_Sety(real_y)
        if epoch >= 1:
            generated_Yset.append(generatedSetY)
        else:
            generatedSetY = generatedSetY[0:100]
        for generatedY in generatedSetY:
            nearestX, nearestY = selectY(x, y, generatedY, 2)
            alpha = Calculated_weight(generatedY, nearestY)# 计算权重
            print("第%s次生成的光谱理化值：%s"%(j, generatedY))
            print('相近的光谱理化值：%s'%nearestY)
            print('在训练中所占权重：%s'%alpha)
            nearestX1 = nearestX[0].reshape(1,2500)
            nearestX2 = nearestX[1].reshape(1,2500)
            start = time.perf_counter()
            for i in range(181):
                codeInputG = inputG(generatedY)# 独热编码+噪声数据
                with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                    # 计算最外层损失
                    outermost_loss1 = outermost_of_G(codeInputG)# G_最外层损失
                    outermost_loss2 = outermost_of_D(nearestX, codeInputG, k)# D_最外层损失
                    # 计算内层损失
                    mesosphere_loss11 = mesosphere_of_G(codeInputG, discriminator1, discriminator4)# G_内层损失
                    mesosphere_loss12 = mesosphere_of_D(codeInputG, nearestX1, discriminator1, discriminator4, k)# D_内层损失
                    mesosphere_loss13 = mesosphere_of_D(codeInputG, nearestX2, discriminator1, discriminator4, k)# D_内层损失
                    mesosphere_loss21 = mesosphere_of_G(codeInputG, discriminator2, discriminator3)# G_内层损失
                    mesosphere_loss22 = mesosphere_of_D(codeInputG, nearestX1, discriminator2, discriminator3, k)# D_内层损失
                    mesosphere_loss23 = mesosphere_of_D(codeInputG, nearestX2, discriminator2, discriminator3, k)# D_内层损失
                    # 计算最里层损失
                    Inner_loss = Innermost_L(nearestX1, nearestX2, codeInputG, discriminator5, generatedY, nearestY)
                    # 总损失
                    gen_loss = outermost_loss1 + mesosphere_loss11 + mesosphere_loss21 + Inner_loss
                    disc_loss = outermost_loss2 + (mesosphere_loss12 + mesosphere_loss22)*alpha + (mesosphere_loss13 + mesosphere_loss23)*(1 - alpha) + Inner_loss 
                # 传入loss和模型参数，计算生成器的权值调整
                gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
                # 传入loss和模型参数，计算判别器的权值调整
                gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
                # 生成器的权值调整
                generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
                # 判别器的权值调整
                discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
                #更新k
                goal,balance = global_process(nearestX, codeInputG,gamma)
                k = tf.compat.v1.assign(k, tf.clip_by_value(k + lambda_k1 * balance, 0, 1))
                image_train(nearestX, codeInputG, i, j, alpha)
                if i/179 == 1:
                    end = time.perf_counter()
                    print('第%s次训练完成，花费时间：%s'%(j, end-start))
                    if epoch >= 1:
                        generation = generator(codeInputG)
                        generated_spectrumSet.append(generation)
                    j = j + 1
        #输出goal
        print('第%s次训练的全局得分：%s'%(epoch, goal))
        #输出全局k
        print('第%s次训练的k值：%s'%(epoch, k))


In [ ]:
with tf.device('/GPU:0'):
    start = time.perf_counter()
    Generated_spectrum()
    end = time.perf_counter()
    print('训练总时长: %s Seconds'%(end-start))

In [ ]:
# 将list转为numpy
print(generated_Yset)

In [ ]:
major_RevsionX2 = np.array(generated_spectrumSet).reshape(600,2500)
major_RevsionY2 = np.array(generated_Yset).reshape(1,600)

In [ ]:
os.chdir(r'D:\Major_revision\Major_RevisondataSet')
np.save('major_RevsionX2.npy',major_RevsionX2)#如果文件路径末尾没有扩展名.npy，该扩展名会被自动加上
np.save('major_RevsionY2.npy',major_RevsionY2)